In [1]:
import pandas as pd
data = pd.read_csv('train.tsv',sep = '\t',header= None)
data

,0,1
0,ansin,N
1,),N
2,tá,N
3,níos,N
4,lú,N
...,...,...
5057054,do,N
5057055,daoine,S
5057056,beaga,N
5057057,.,N


In [2]:
data.columns = ['Token','mutation']

In [3]:
list_add=[]
length= data.shape[0]
temp=2
while temp!= data.shape[0]:
  list_add.append( str(data['Token'][temp-2]) +' '+ str(data['Token'][temp-1])+' '+ str(data['Token'][temp]))
  temp+=1
list_add.insert(0,'ansin')
list_add.insert(1, 'ansin )')

In [4]:
mut = []
length= data.shape[0]
temp=0
while temp!= data.shape[0]:
  mut.append(data['mutation'][temp])
  temp+=1

In [5]:
tokens = { 'Mutations': mut,
            'Tri-gram' : list_add
    
}
tokens = pd.DataFrame(tokens)
tokens

,Mutations,Tri-gram
0,N,ansin
1,N,ansin )
2,N,ansin ) tá
3,N,) tá níos
4,N,tá níos lú
...,...,...
5057054,N,tír mór do
5057055,S,mór do daoine
5057056,N,do daoine beaga
5057057,N,daoine beaga .


In [7]:
tokens['Tri-gram'] = tokens['Tri-gram'].str.replace("[<URI><URL><USER><EMAIL><NUM>]","")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [8]:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
tokens['Mutations'] = le.fit_transform(tokens['Mutations'])

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

count = CountVectorizer(max_features=1500)
input_text = count.fit_transform(tokens['Tri-gram'])

x_train_cv,x_test_cv,y_train_cv,y_test_cv = train_test_split(input_text,tokens['Mutations'],test_size=0.1)

In [12]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(x_train_cv,y_train_cv)

from sklearn.metrics import accuracy_score
pred = tree.predict(x_test_cv)
accuracy_score(y_test_cv,pred)

0.8938889394233013

In [9]:
import nltk
import keras
import tensorflow
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize

In [10]:
from sklearn.model_selection import train_test_split
x = tokens['Tri-gram']
y = tokens['Mutations']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.1, random_state=41)

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

In [12]:
vocab = data['Token'].tolist()

In [13]:
number_words = len(list(set(vocab)))
number_words

126548

In [14]:
# tokenizing all tweets 
tokenizer = Tokenizer(num_words=number_words)
tokenizer.fit_on_texts(x_train)

train_sequence = tokenizer.texts_to_sequences(x_train)

test_sequence = tokenizer.texts_to_sequences(x_test)

In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_train = pad_sequences(train_sequence,maxlen=2, padding = 'post')
padded_test = pad_sequences(test_sequence,maxlen=2,padding = 'post')

In [16]:
y_train_cat = to_categorical(y_train,5)
y_test_cat = to_categorical(y_test,5)

In [17]:
#LSTM model using keras library

model = Sequential()

model.add(Embedding(number_words, output_dim=128))

model.add(LSTM(256))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5,activation='softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         16198144  
                                                                 
 lstm (LSTM)                 (None, 256)               394240    
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 5)                 325       
                                                                 
Total params: 16,609,157
Trainable params: 16,609,157
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(padded_train,y_train_cat, batch_size= 1024, epochs= 3, validation_split= 0.1)

Epoch 1/3
4001/4001 [==============================] - 1154s 288ms/step - loss: 0.1795 - accuracy: 0.9371 - val_loss: 0.1388 - val_accuracy: 0.9478
Epoch 2/3
4001/4001 [==============================] - 1144s 286ms/step - loss: 0.1270 - accuracy: 0.9519 - val_loss: 0.1358 - val_accuracy: 0.9491
Epoch 3/3
4001/4001 [==============================] - 1149s 287ms/step - loss: 0.1168 - accuracy: 0.9551 - val_loss: 0.1377 - val_accuracy: 0.9492


In [19]:
pred = model.predict(padded_test)

In [20]:
import numpy as np
predictions = []
for i in pred:
  predictions.append(np.argmax(i))

original = []
for i in y_test_cat:
  original.append(np.argmax(i))

In [21]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(original,predictions)
accuracy

0.9491878680498155